In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets/100k_tweets_and_encodings_min_1_emoji_from_top100.csv
/kaggle/input/tweets/tweets_fr.txt
/kaggle/input/tweets/1M_tweets_and_encodings.csv/1M_tweets_and_encodings.csv


In [2]:
NROWS = 100000
df = pd.read_csv('/kaggle/input/tweets/100k_tweets_and_encodings_min_1_emoji_from_top100.csv', nrows=NROWS)

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df


,original_tweet,clean_tweet,emoji_0,emoji_1,emoji_2,emoji_3,emoji_4,emoji_5,emoji_6,emoji_7,...,emoji_90,emoji_91,emoji_92,emoji_93,emoji_94,emoji_95,emoji_96,emoji_97,emoji_98,emoji_99
0,@chlechevalier @agnesbuzyn @googlenews macron ...,macron n a pas honte de faire la fête brizitte...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,et vous ? 🥰 bonne année les amis ! 💃 twitterli...,et vous bonne année les amis twitterlink,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,villa brûlée ✅ bientôt la venue de mujdat 🙃 #l...,villa brûlée bientôt la venue de mujdat lmvsmo...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,mary poppins - supercalifragilisticexpialidoci...,mary poppins supercalifragilisticexpialidociou...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,le seul truc qui me choque dans house of cards...,le seul truc qui me choque dans house of cards...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,@lindalovefoxxx ooooh chet 🤤 🤤 👅 👅 👅 👅 🍆\n,ooooh chet,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,@fortnitefr et les champions de drogue c ' est...,et les champions de drogue c est autorisé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,[ #brestatlantiques ] 🤗 h - 1 avant le grand d...,brestatlantiques h 1 avant le grand départ de ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,gagner 230e au casino pour bien commencer l ’ ...,gagner 230e au casino pour bien commencer l année,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
!pip install omegaconf
!pip install hydra-core
!pip install torch 


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 122 kB 861 kB/s 
     |████████████████████████████████| 112 kB 3.3 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=ec8d44a3474749f0d3a95274eaf6642c472c73a54b601bbf8d453da52cc591d6
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
import torch
camembert = torch.hub.load('pytorch/fairseq', 'camembert')
camembert.eval()  # disable dropout (or leave in train mode to finetune)

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp
cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating /root/.cache/torch/hub/pytorch_fairseq_master/build
creating /root/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7
creating /root/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/fairseq
creating /root/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/fairseq/clib
creating /root/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
Emitting ninja build file /root/.cache/torch/hub/pytorch_fairseq_master/build/temp.linux-x86_64-3.7/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/fairseq
g++ -pth

100%|██████████| 1012630426/1012630426 [01:31<00:00, 11015397.24B/s]


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(32005, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=T

In [6]:
masked_line = "J'aime me beurrer la <mask> "
camembert.fill_mask(masked_line, topk=10)

/root/.cache/torch/hub/pytorch_fairseq_master/fairseq/models/roberta/hub_interface.py:176: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729141890/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  masked_index = (tokens == self.task.mask_idx).nonzero()


[("J'aime me beurrer la joue ", 0.11025537550449371, ' joue'),
 ("J'aime me beurrer la bouche ", 0.0751555860042572, ' bouche'),
 ("J'aime me beurrer la peau ", 0.05565204098820686, ' peau'),
 ("J'aime me beurrer la main ", 0.04818896949291229, ' main'),
 ("J'aime me beurrer la moustache ", 0.0477130189538002, ' moustache'),
 ("J'aime me beurrer la nuque ", 0.03552733734250069, ' nuque'),
 ("J'aime me beurrer la cuisse ", 0.022842274978756905, ' cuisse'),
 ("J'aime me beurrer la tronche ", 0.022781599313020706, ' tronche'),
 ("J'aime me beurrer la moule ", 0.02265654131770134, ' moule'),
 ("J'aime me beurrer la cervelle ", 0.022106261923909187, ' cervelle')]

In [7]:
line = "J'aime le camembert !"
tokens = camembert.encode(line)
last_layer_features = camembert.extract_features(tokens, return_all_hiddens=False)
last_layer_features.size()

torch.Size([1, 10, 768])

In [8]:
clean_tweets = df['clean_tweet'].values
clean_tweets=clean_tweets.astype('str')


In [9]:
last_layer_features.size()
token_embeddings = torch.squeeze(last_layer_features, dim=0)
token_embeddings.size()
sentence_embedding = torch.mean(token_embeddings, dim=0)
sentence_embedding.size()

torch.Size([768])

In [10]:
from tqdm.notebook import tqdm as tqdm 


In [11]:
clean_tweets = df['clean_tweet'].values.astype('str')
#print(type(clean_tweets[0]))
empty = []
sentence_embeddings = np.array(empty)
len(clean_tweets)
for tweet in tqdm(clean_tweets) :
    tokens = camembert.encode(tweet)
    last_layer_features = camembert.extract_features(tokens) # On prend la dernière couche de BERT pour chacun des tokens 
    token_embeddings = torch.squeeze(last_layer_features, dim=0) # On enlève la dimension inutile qui vaut 1 car on n'a qu'une phrase
    sentence_embedding = torch.mean(token_embeddings, dim=0).detach().numpy() # On fait la moyenne sur tous les tokens pour "projeter" i.e passer de [n_tokens,768] à [768] 
    sentence_embeddings = np.append(sentence_embeddings, sentence_embedding)
#stacked_encodings = np.stack(sentence_embeddings)
    

In [12]:
sentence_embeddings = sentence_embeddings.reshape(NROWS,768)
sentence_embeddings.shape


(100000, 768)

In [13]:
np.savetxt('100k_sentence_embeddings_min_1_emoji_from_top100.csv', sentence_embeddings, delimiter=',')